In [ ]:
import csv
csv_path="raw_aozora/list_person_all_extended_utf8.csv"
with open(csv_path,'r',encoding='UTF-8') as f:
    reader = csv.reader(f)
    csv_list=[row for row in reader]

In [ ]:
#auther_idとnovel_idの辞書を作成(一対多)
#辞書の使い勝手が良いので一度辞書にしている。
auther_id_and_novel_id_dict={}
for csv_attr in csv_list[:]:
    if csv_attr[9]=="新字新仮名":#'文字遣い種別'
        if csv_attr[10]=="なし":#'作品著作権フラグ'
            auther_id=csv_attr[14]#人物ID
            novel_id=str(int(csv_attr[0]))#小説ID先頭の0を消しておく
            auther_id_and_novel_id_dict.setdefault(auther_id, []).append(novel_id)
print('end')

In [ ]:
#auther_and_novel_dictをauther_id_listとnovel_id_listに分割
#リストは小説名などに用いやすい。
auther_id_list=[]
novel_id_list=[]
for auther_id in auther_id_and_novel_id_dict:
    auther_id_list.append(auther_id)
    novel_id_list.append(sorted(auther_id_and_novel_id_dict[auther_id]))

In [ ]:
#青空文庫のcardの中から本文を含むファイルのパスをリストに入れる
#参考 https://serenard.hatenablog.com/entry/2019/04/07/164337
import glob
cards_path="raw_aozora/aozorabunko/cards"
novel_html_paths=[]
for enr,auther_id in enumerate(auther_id_list[:]):
    for novel_id in novel_id_list[enr]:
        novel_html_path = glob.glob(f'{cards_path}/{auther_id}/files/{novel_id}_*.html')
        novel_html_paths.append(novel_html_path)

In [ ]:
from bs4 import BeautifulSoup as soup
#ファイルパスを用いてローカルでbs4でのスクレイピングを行う
sav_p=0 #値変更で途中から開始可能
all_novel_text=""
for enr,novel_html_path in enumerate(novel_html_paths[sav_p:],sav_p):
    if novel_html_path==[]:#htmlが無いものは省く。現在作業中のものは無いようだ。
        continue
    with open(novel_html_path[0],'r',encoding='shift-jis') as f:
        try:
            novel_page_html = f.read()
        except UnicodeDecodeError:#エラーの性質上、扱いにくいのでskip。
            continue
    novel_soup = soup(novel_page_html,'html.parser')
    for rt in novel_soup('rt'):#ルビ削除
        rt.decompose()
    for rp in novel_soup('rp'):#ルビ削除
        rp.decompose()
    novel_content = novel_soup.find('div', class_='main_text')
    if novel_content is not None:
        all_novel_text+=novel_content.text
    if enr%1000==0:
        print(enr)
print(enr,'end')

In [ ]:
#shift-jisで一度保存する
with open("aozora_main_shift_jis.txt",'w',encoding='shift-jis') as f:
    f.write(all_novel_text)

In [ ]:
#utf-8のもので作り直す
import codecs
with codecs.open('aozora_main_shift_jis.txt', 'r', encoding='shift-jis')as f:
    read_shift_jis=f.read()
with codecs.open('aozora_main_utf8.txt', 'w', encoding='utf-8')as f:
    f.write(read_shift_jis)